In [ ]:
import glob
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from netCDF4 import Dataset

# 获取所有符合条件的 nc 文件
nc_files = glob.glob('/shared1/RESEARCH_DATA/GRFR/output_pfaf_*_1979-2019.nc')
# 遍历 '01' 到 '08'
for i in range(1, 9):
    prefix = f"{i:02d}"  # 格式化成 '01', '02', ..., '08'
    nc_filename = f"/shared1/RESEARCH_DATA/GRFR/output_pfaf_{prefix}_1979-2019.nc"
    
    print(f"正在处理文件: {nc_filename}")
    
    f = Dataset(nc_filename)
    
    comid = f.variables['rivid'][:]
    qout = f.variables['Qout'][:]  # Qout shape: (time, comid)
    q50 = np.median(qout, axis=0)  # 按时间维度计算中位数
    
    df = pd.DataFrame(data={'COMID': comid, 'q50': q50})
    
    output_csv = f"2.GRFR_q50_{prefix}.csv"
    df.to_csv(output_csv, index=False)
    
    print(f"已保存为: {output_csv}")

正在处理文件: /shared1/RESEARCH_DATA/GRFR/output_pfaf_01_1979-2019.nc


In [ ]:
import os
# 获取所有匹配的 nc 文件
csv_files = glob.glob('2.GRFR_q50_0*.csv')

# 用于存储多个df
dfs = []

# 遍历所有nc文件
for file_path in csv_files:
    filename = os.path.basename(file_path)
    
    # 提取 pfaf 编号，如 output_pfaf_07_1979-2019.nc -> 07
    pfaf = filename.split('_')[2].split('.')[0]
    df = pd.read_csv(file_path)
    dfs.append(df)


# 合并所有 DataFrame
final_df = pd.concat(dfs, ignore_index=True)
gages_3000 = pd.read_csv('1.GAGE_3000pointsbuffer_GLOW_points_matched.csv')
df_merged = gages_3000.merge(final_df,on='COMID',how='left')

# 保存合并后的数据
df_merged.to_csv('2.gages3000_GRFR_q50.csv', index=False)
print("合并完成，已保存为 merged_q50_all.csv")
print(df_merged.columns)

In [ ]:
print(df_merged.head())
print(df_merged['stationid'].nunique())